<a href="https://colab.research.google.com/github/reidnersousa/IFB_Arquitetura-de-Computadores-II_2023-1/blob/main/trabalho_Arquitetura_questao_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

(1,0 ponto) Crie um programa em CUDA C  que cria três vetores: b, c e d, depois executa (de forma vetorial, usando a GPU) a seguinte operação:
a[i] = (b[i] + c[i]) * (d[i] / c[i]) - 50);
Por fim, exiba os valores dos vetores a, b, c e d.


Onde se lê: a[i] = (b[i] + c[i]) * (d[i] / c[i]) - 50);


Leia-se: a[i] = (b[i] + c[i]) * ((d[i] / c[i]) - 50);

In [35]:
%%writefile questao1.cu

#include<stdio.h>
#include<time.h>
#include<stdlib.h>




__global__ void add(int *a, int *b, int *c, int *d) {
	// para resolver o problema  de divisao


	a[blockIdx.x] = (b[blockIdx.x] + c[blockIdx.x]) * ((float)d[blockIdx.x] / c[blockIdx.x]) - 50; // usando esse a reposta tem uma precisao melhor
//  a[blockIdx.x] = (b[blockIdx.x] + c[blockIdx.x]) * ((d[blockIdx.x] / c[blockIdx.x]) -50) ;  // esse aqui ta tendo erro de precisao

}

void random_ints(int *a, int N){
	int i;
	for (i = 0; i < N; i++){
		a[i] = rand() % 100;
		printf("%3d ", a[i]);
	}
	puts("");
}

#define N 1000

int main(void) {
	int *a, *b, *c, *d; // host copies of a, b, c, d
	int *d_a, *d_b, *d_c, *d_d; // device copies of a, b, c, d
	int size = N * sizeof(int);
	int i;

	// Allocate space for device copies of a, b, c, d
	cudaMalloc((void **)&d_a, size);
	cudaMalloc((void **)&d_b, size);
	cudaMalloc((void **)&d_c, size);
	cudaMalloc((void **)&d_d, size);

	srand(time(NULL));

	// Allocate space for host copies of b, c, d and setup input values
	printf("b");
	b = (int *)malloc(size); random_ints(b, N);
	printf("c");
	c = (int *)malloc(size); random_ints(c, N);
	printf("d");
	d = (int *)malloc(size); random_ints(d, N);
	a = (int *)malloc(size);

	cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);
	cudaMemcpy(d_c, c, size, cudaMemcpyHostToDevice);
	cudaMemcpy(d_d, d, size, cudaMemcpyHostToDevice);

	add<<<N, 1>>>(d_a, d_b, d_c, d_d);

	cudaMemcpy(a, d_a, size, cudaMemcpyDeviceToHost);

	for (i = 0; i < N - 1; i++){
		printf(">>%d,", a[i]);
	}
	printf(">>%d\n", a[i]);

	free(a); free(b); free(c); free(d);
	cudaFree(d_a); cudaFree(d_b); cudaFree(d_c); cudaFree(d_d);

	return 0;
}


Overwriting questao1.cu


In [36]:
!nvcc questao1.cu -o questao1
!./questao1

b 50  13   0  35  84  24  87  84  33  89  15  82  15  17  90  36  91  79  38  79  52  65  71  75  16  68  23   6  59  74  77  61  39  77  96  76  54  35  12  87  76  27  22  43  45  12  79  88  92  70  19  96  35  43  71   3  63  95  62  22  21  39  83  12  68  31  88  22  18   0  62  94  80  84  37  77  96  68  65  40  38  36  37  26  79   8  29  94  55  91  16  28  30  51  93  51  34  33  73  52  86  35  98  66  71  87  43  68  56  60  60  46  96  97  72  28  58  54  22  65  45  91  94  28  94  87  79  29  72   4  33  58  92  84  24  63  71  19  83  79  79  44  26  28  41  50  56  51   4  30  17  50  73  63  30  68   2  61  49  74  65  34  33   9  18  57  73  90  77  56  21   8  52  47  36  94  50  44  97  54  75  14  56  48  29  38  68  31  99  17  58  17   4  91  26  22   0  51  64  29   8  86  38  60  85  26   6  35  71  56  90  98  70  98  98  52  37  67  83  36  36  93  53  40  84  80  15  37  83  79  66  91  65  56   4   3  35  62  38  58  18  80  56  41  79   6  45  16  73  80

In [69]:
import numpy as np
#                       (b[i] + c[i]) * (d[i] / c[i]) - 50;

a =           np.float64(50.0 + 44.0) * ((65.0 / 44.0) - 50.0)
aSemDivProb = np.float64(50.0 + 44.0) * (65.0 / 44.0) - 50.0

print(a)
print(aSemDivProb)


-4561.136363636364
88.86363636363637


Usando float

In [27]:
%%writefile questao1_com_float.cu
#include <stdio.h>
#include <time.h>
#include <stdlib.h>

__global__ void add(float *a, int *b, int *c, int *d) {
    a[blockIdx.x] = (b[blockIdx.x] + c[blockIdx.x]) * ((float)d[blockIdx.x] / c[blockIdx.x]) - 50.0f;
}

void random_ints(int *a, int N) {
    int i;
    for (i = 0; i < N; i++) {
        a[i] = rand() % 100;
        printf("%3d ", a[i]);
    }
    puts("");
}

#define N 1000

int main(void) {
    float *a; // Change the data type to float
    int *b, *c, *d;
    float *d_a; // Change the data type to float
    int *d_b, *d_c, *d_d;
    int size = N * sizeof(int);
    int i;

    // Allocate space for device copies of a, b, c, d
    cudaMalloc((void **)&d_a, N * sizeof(float)); // Change the data type to float
    cudaMalloc((void **)&d_b, size);
    cudaMalloc((void **)&d_c, size);
    cudaMalloc((void **)&d_d, size);

    srand(time(NULL));

    // Allocate space for host copies of b, c, d and setup input values
    printf("b");
    b = (int *)malloc(size); random_ints(b, N);
    printf("c");
    c = (int *)malloc(size); random_ints(c, N);
    printf("d");
    d = (int *)malloc(size); random_ints(d, N);
    a = (float *)malloc(N * sizeof(float)); // Change the data type to float

    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_c, c, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_d, d, size, cudaMemcpyHostToDevice);

    add<<<N, 1>>>(d_a, d_b, d_c, d_d);

    cudaMemcpy(a, d_a, N * sizeof(float), cudaMemcpyDeviceToHost); // Change the data type to float

    for (i = 0; i < N - 1; i++) {
        printf(">>%.2f,", a[i]); // Print the values as floating-point numbers with 2 decimal places
    }
    printf(">>%.2f\n", a[i]); // Print the last value as a floating-point number with 2 decimal places

    free(a); free(b); free(c); free(d);
    cudaFree(d_a); cudaFree(d_b); cudaFree(d_c); cudaFree(d_d);

    return 0;
}


Writing questao1_com_float.cu


In [29]:
!nvcc questao1_com_float.cu -o questao1
!./questao1

b 61  21  62   3  78  82  42  81  26  65  72  37   8  51  31  96  95  90  73  86  37  43  24  38  66  29  73  20   7  61   0  69  82  63  72  12  97  14  93  75  32  18  12  92  69  44  88  16  86  62  54  23  57  31  13  76  60  86  48  67  99  49  36  81  64  61  45  61  75  39  36   7   9   0   0  30  44  40  46  30  54  53   5  12  84  18  88  44  57  88  63   8  37  52  42   1  13  87  14  40  78   2  48  87   3   0  69  99  92  16  82  47  21  87  11   5  58  51   1  67  39  64  75  77  16  69  30  81  57  45  22  35  99  22  75   2  74  44   2  66  12  36  65  33  23  76  38  33  27  39   0  67  56  28  96  24  49  78   6   6  23  80  94  23  54  69  77  28  65  31  94  30  67  12  63  43  88  54  76  68  45  29  35  53   9  83  78  58  61  36  17  37  68  11  12  22  32  41  50  97  73  96  27  92   8  43  35  97  97  64  65  94  45  52  48  54  87  26  64   0  14  81  37  82  92   1   4  24  43  54  74  68   2  53  12  11  96   0  60  45  16  77  40  61  81  88  15  68  66  31

In [37]:
import numpy as np
#                       (b[i] + c[i]) * (d[i] / c[i]) - 50;

a =           np.float64(50.0 + 44.0) * ((65.0 / 44.0) - 50.0)
aSemDivProb = np.float64(50.0 + 44.0) * (65.0 / 44.0) - 50.0

print(a)
print(aSemDivProb)


-4561.136363636364
88.86363636363637


Crie um programa em CUDA C que implementa os códigos abaixo usando uma GPU e imprima os valores dos vetores ao final de cada execução:
 (1,0 ponto)
for (int i = 0; i < N; i++){
	if (vetor2[i] > 0){
		vetor2[i] = vetor2[i] * -1;
	}
	else{
	vetor2[i] = vetor2[i] * vetor1[i];
	}
}


for (int i = 0; i < N; i++){
	if (vetor2[i] > 0){
		vetor2[i] = vetor2[i] * -1;
	}
	else{
	vetor2[i] = vetor2[i] * vetor1[i];
	}
}


In [ ]:
"""
for (int i = 0; i < N; i++){
	if (vetor2[i] > 0){
		vetor2[i] = vetor2[i] * -1;
	}
	else{
	vetor2[i] = vetor2[i] * vetor1[i];
	}
} """


In [42]:


%%writefile letraA.cu


// verifica se ta certo

#include<stdio.h>
#include<time.h>
#include<stdlib.h>

__global__ void add(int *vetor1, int *vetor2) {
    int index = threadIdx.x + blockIdx.x * blockDim.x;
    if (vetor2[index] > 0) {
        vetor2[index] = vetor2[index] * -1;
    }
    else{
      vetor2[index] =vetor2[index]*vetor1[index];
    }
}

void random_ints(int *a, int N){
	int i;
	for (i = 0; i < N; i++){
      a[i] = (rand() % 100) - 100;

	  // a[i] = rand() % 100;
    printf("%3d ",a[i]);
	}
  puts("");
}



#define N (1000)
#define THREADS_PER_BLOCK 8

int main(void) {
	int *vetor1, *vetor2; // host copies of a, b, c
    int *d_vetor1, *d_vetor2; // device copies of a, b, c
    int size = N * sizeof(int);
    int i;

    // Alloc space for device copies of a, b, c
    cudaMalloc((void **)&d_vetor1, size);
    cudaMalloc((void **)&d_vetor2, size);

    srand(time(NULL));

    // Alloc space for host copies of a, b, c and setup input values
    vetor1 = (int *)malloc(size);
    random_ints(vetor1, N);
    vetor2 = (int *)malloc(size);
    random_ints(vetor2, N);

    // Copy inputs to device
    cudaMemcpy(d_vetor1, vetor1, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_vetor2, vetor2, size, cudaMemcpyHostToDevice);

    // Launch add() kernel on GPU with N blocks
    add<<<((N + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_vetor1, d_vetor2);

    // Sincronização do dispositivo
    cudaDeviceSynchronize();

    // Copy result back to host
    cudaMemcpy(vetor2, d_vetor2, size, cudaMemcpyDeviceToHost);

    for (i = 0; i < N - 1; i++) {
        printf("%-3d,", vetor2[i]);
    }
    printf("%-3d\n", vetor2[i]);

    // Cleanup
    free(vetor1);
    free(vetor2);
    cudaFree(d_vetor1);
    cudaFree(d_vetor2);

	  return 0;
}


Overwriting letraA.cu


In [43]:
!nvcc letraA.cu -o letraA
!./letraA

-45 -99 -15 -29 -90 -15 -40 -79 -85 -74 -51 -85 -28 -24 -50 -42 -29 -54 -59 -18  -9 -30  -9 -71 -69 -22 -54 -65 -78 -80 -10 -71 -79 -25 -48 -17 -88 -88 -96 -21  -9 -47  -6 -85 -18 -56 -27 -47 -58 -34 -65 -14 -64 -22 -85 -33 -44 -86 -46 -70 -66 -56 -41 -92 -29 -88  -9 -16 -24 -52 -85 -81 -47 -90 -66 -65 -94 -41 -60 -99 -75 -73 -13 -86 -94 -98 -67 -38 -84 -12 -55 -97 -16 -96 -89 -92 -32 -46 -56  -3 -98 -41 -84 -44 -79 -97 -57 -20 -86 -16 -19  -8 -89 -32 -42 -31 -78 -57 -68 -61 -17 -23 -58 -32 -67 -95 -72 -46 -41 -28 -49 -86 -17 -81 -78 -95 -26 -35 -15 -11 -99 -82 -19 -35 -14  -9 -66 -92 -66 -82  -1 -82 -53  -7 -62 -67  -2 -34 -13 -90 -10 -62 -76 -27 -91  -2 -70 -16 -37 -33 -75 -83 -15 -42 -18 -29 -51 -32 -68 -17 -13 -69 -47 -14 -75  -9 -81 -25 -43 -94 -15 -53  -4 -39 -27 -42 -41 -97 -58 -25 -29 -81 -56 -92 -23 -22 -20 -22 -54 -88 -87 -15  -5 -33 -76 -28 -42 -57 -52 -32 -98 -15 -85  -2  -1 -60 -44 -90 -56 -50 -63 -33 -31 -19 -25  -2 -41 -93 -24 -42 -29 -58 -57 -33 -91 -33 -61 -33 -37 -61 

In [ ]:
int j = 0;
for (int i = 32; i >= 0;  i--){
	vetor2[j] =  vetor1[i];
	j++;
}


In [ ]:
%%writefile letraB.cu

// certo  B
#include<stdio.h>
#include<time.h>
#include<stdlib.h>

__global__ void add(int *vetor1, int *vetor2) {

    int index = threadIdx.x + blockIdx.x * blockDim.x;
    int jndex = blockDim.x * blockIdx.x + (blockDim.x - threadIdx.x - 1);
    vetor2[jndex] = vetor1[index];

}

void random_ints(int *a, int N){
	int i;
	for (i = 0; i < N; i++){
     a[i] = (rand() % 100) ;

	// a[i] = rand() % 100;
    printf("%3d ",a[i]);
	}
  puts("");
}


# define  QTD 1000
#define N (QTD)
#define THREADS_PER_BLOCK QTD

int main(void) {
	int *vetor1, *vetor2; // host copies of a, b, c
    int *d_vetor1, *d_vetor2; // device copies of a, b, c
    int size = N * sizeof(int);
    int i;

    // Alloc space for device copies of a, b, c
    cudaMalloc((void **)&d_vetor1, size);
    cudaMalloc((void **)&d_vetor2, size);

    srand(time(NULL));

    // Alloc space for host copies of a, b, c and setup input values
    vetor1 = (int *)malloc(size);
    random_ints(vetor1, N);
    vetor2 = (int *)malloc(size);
    random_ints(vetor2, N);

    // Copy inputs to device
    cudaMemcpy(d_vetor1, vetor1, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_vetor2, vetor2, size, cudaMemcpyHostToDevice);

    // Launch add() kernel on GPU with N blocks
    add<<<((N + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_vetor1, d_vetor2);

    // Sincronização do dispositivo
    cudaDeviceSynchronize();

    // Copy result back to host
    cudaMemcpy(vetor2, d_vetor2, size, cudaMemcpyDeviceToHost);

    for (i = 0; i < N - 1; i++) {
        printf("%-3d,", vetor2[i]);
    }
    printf("%-3d\n", vetor2[i]);

    // Cleanup
    free(vetor1);
    free(vetor2);
    cudaFree(d_vetor1);
    cudaFree(d_vetor2);

	  return 0;
}



Overwriting letraB.cu


In [ ]:
!nvcc letraB.cu -o letraB
!./letraB

 58   2  36  55  13  55  11  74  84  48  76  29  64  23   3   2  36  44  97  32  11  31  50  55  83  11  94  60  67  44   3  26  46  39  81  12  95  44  38  31  44  14  60   8  89  63  63  78  59  60  10  70  43  61  26  26  24  72  39  91  16  42  69  14  81   2  26  28  47  64  59  91  31  19  52  72  34  15   2  93  27  13  64  71  26  42  49   2  14  88  45  82  30  15  48  64  17  75  92  64  91   4   8  74  75  12  47  10  27  49   3  54  14  19  77  92  13  79  94  27  67  92  61  50   7  10  66  76  37  10  93  80  14  53  55  90  65  54  52  44  55  55  50  70  27  28  14  40   7   9  20  26  53  81  28  12  43  94  88  80  57  81  61  23  86  68  65  51  22  17  95  29  25  46  51  52  26  66  92  85  27  12  63  80  46  92  92  89  38  32  22  95  66  35  71  52   3  36  56  77  54  51   6  79  49  58  83  75  76  75  12   3  40  76  35  38  68  79  27  58  11   1   6  77  36  77  82  39  13  38  68  19  89  27  50  91  37  85  66  13  13  79  68  53  55   3  91  75  82  70 

In [ ]:
for (i = 0; i < N-1; i++){
a[i] += a[i + 1];
}
a[i] = a[i]+ a[i+1];


In [ ]:
%%writefile letrac.cu

// c certo
#include<stdio.h>
#include<time.h>
#include<stdlib.h>

__global__ void add(int *a, int N) {
    int index = threadIdx.x + blockIdx.x * blockDim.x;
    if (index < N - 1) {
        a[index] += a[index + 1];
    }
}

void random_ints(int *a, int N){
	int i;
	for (i = 0; i < N; i++){
     a[i] = (rand() % 100) ;

	// a[i] = rand() % 100;
    printf("%3d ",a[i]);
	}
  puts("");
}


# define  QTD 1000
#define N (QTD)
#define THREADS_PER_BLOCK QTD

int main(void) {
	int *vetor1; // host copies of a, b, c
    int *d_vetor1;  // device copies of a, b, c
    int size = N * sizeof(int);
    int i;

    // Alloc space for device copies of a, b, c
    cudaMalloc((void **)&d_vetor1, size);


    srand(time(NULL));

    // Alloc space for host copies of a, b, c and setup input values
    vetor1 = (int *)malloc(size);
    random_ints(vetor1, N);

    // Copy inputs to device
    cudaMemcpy(d_vetor1, vetor1, size, cudaMemcpyHostToDevice);


    // Launch add() kernel on GPU with N blocks
    add<<<((N + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_vetor1,N);

    // Sincronização do dispositivo
    cudaDeviceSynchronize();

    // Copy result back to host
    cudaMemcpy(vetor1, d_vetor1, size, cudaMemcpyDeviceToHost);

    for (i = 0; i < N - 1; i++) {
        printf("%-3d,", vetor1[i]);
    }
    printf("%-3d\n", vetor1[i]);

    // Cleanup
    free(vetor1);

    cudaFree(d_vetor1);


	  return 0;
}



Overwriting letrac.cu


In [ ]:
!nvcc letrac.cu -o letrac
!./letrac

 95  58  54  41  53  20  77  59  82  25   4  73  86  76  38   8  68  30  54  72  52   5  47  49  19  48  73  75  17  59  35  64  69  89  57  23  10  34  34  92  11  38  17  97  66  56  57  35  38  12  59  90  69   7  91  89  55  64  16  24  76  51  88  45  93  45  20   3  31   6  47  95  97  17  92  63  73   2  98  11  66  10  53  87  17  45  28  24  61  44  49  89  96  89  35  41  35   7  96  18  14  43  13  11  12  58  74  85  12  25  48  78  35   2  17   4  99  46  80  12  90  29   2  38  19  89  79   6  96  75  76  10  71  42  73  35  52  48  21  64  25  69  94  12  23  11  16  22   9  48  35   0  78  37  90  49  26  22  55  74  49  83  85  72  25  58   8  77  58  29  93  83  50  87  47  74  51  15  48  60  64  83  12  42  72  55  43  50  77  98  25  78  33  62  51  59  72  59  88  83  40  82  18  42  21  66  68  24  81  17  37  45  52   1  39  77  56  82  27  85  32   4  64  66  18  15  77  43  26  65  26  18  99  96  12  73  62  81  97  96  98  34  41  50  36  33  27  44  15   7 

letra D    onde se lê:  a[i] = a[N - i + 1];



leia-se: a[i] = a[N - i - 1];



In [ ]:
 for (i = 0; i < N/2; i++){
	a[i] = a[N- i-1];
}


In [66]:
%%writefile letrad.cu

// D
#include<stdio.h>
#include<time.h>
#include<stdlib.h>

__global__ void add(int *a, int N) {
    int index = threadIdx.x + blockIdx.x * blockDim.x;
    if (index < N / 2) {
        a[index] += a[N-index - 1];
    }
    else{
      a[index] +=  a[N-index -1];
    }
}

void random_ints(int *a, int N){
	int i;
	for (i = 0; i < N; i++){
     a[i] = (rand() % 100) ;

	// a[i] = rand() % 100;
    printf("%3d ",a[i]);
	}
  puts("");
}


# define  QTD 10
#define N (QTD)
#define THREADS_PER_BLOCK QTD

int main(void) {
	int *vetor1; // host copies of a, b, c
    int *d_vetor1;  // device copies of a, b, c
    int size = N * sizeof(int);
    int i;

    // Alloc space for device copies of a, b, c
    cudaMalloc((void **)&d_vetor1, size);


    srand(time(NULL));

    // Alloc space for host copies of a, b, c and setup input values
    vetor1 = (int *)malloc(size);
    random_ints(vetor1, N);

    // Copy inputs to device
    cudaMemcpy(d_vetor1, vetor1, size, cudaMemcpyHostToDevice);


    // Launch add() kernel on GPU with N blocks
    add<<<((N + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_vetor1,N);

    // Sincronização do dispositivo
    cudaDeviceSynchronize();

    // Copy result back to host

    cudaMemcpy(vetor1, d_vetor1, size, cudaMemcpyDeviceToHost);

    for (i = 0; i < N - 1; i++) {
        printf("%-3d,", vetor1[i]);
    }
    printf("%-3d\n", vetor1[i]);

    // Cleanup
    free(vetor1);

    cudaFree(d_vetor1);


	  return 0;
}

Overwriting letrad.cu


In [67]:
!nvcc letrad.cu -o letrad
!./letrad

 61  39  60  54  35  87  36  96  37  91 
152,76 ,156,90 ,122,122,90 ,156,76 ,152


In [68]:
import numpy as np

N =10
a = np.array( [59 ,62 ,43, 68, 19 ,19 ,68 ,43 ,62 ,59])

for i in range(N//2):
    a[i] = a[N - i - 1]

print(a)


[59 62 43 68 19 19 68 43 62 59]
